In [1]:
fasta_data = """>sp|P01308|INS_HUMAN Insulin OS=Homo sapiens OX=9606 GN=INS PE=1 SV=1
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAED
LQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
"""

# Step 1: split lines
lines = fasta_data.strip().split("\n")

# Step 2: remove FASTA header (lines starting with '>')
sequence_lines = [line for line in lines if not line.startswith(">")]

# Step 3: join lines
sequence = "".join(sequence_lines)

# Step 4: convert to uppercase
sequence = sequence.upper()

print("Curated sequence:")
print(sequence)


Curated sequence:
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN


In [2]:
valid_amino_acids = set("ACDEFGHIKLMNPQRSTVWY")

def branch_and_bound_kmers(sequence, k):
    result = []

    def backtrack(start, current):
        # BOUND 1: length exceeded
        if len(current) > k:
            return

        # BOUND 2: invalid character
        if any(ch not in valid_amino_acids for ch in current):
            return

        # If k-mer completed
        if len(current) == k:
            result.append(current)
            return

        # BOUND 3: not enough characters left
        if start >= len(sequence):
            return

        # BRANCH: extend substring
        backtrack(start + 1, current + sequence[start])

    # Start branching from every position
    for i in range(len(sequence)):
        backtrack(i, "")

    return result


# Example run
sequence = "MALWMRLLPLLALLALWGPDPAAA"
k = 3

kmers = branch_and_bound_kmers(sequence, k)
print("Consistent k-mers:")
for kmer in kmers:
    print(kmer)


Consistent k-mers:
MAL
ALW
LWM
WMR
MRL
RLL
LLP
LPL
PLL
LLA
LAL
ALL
LLA
LAL
ALW
LWG
WGP
GPD
PDP
DPA
PAA
AAA


In [3]:
from collections import defaultdict

valid_amino_acids = set("ACDEFGHIKLMNPQRSTVWY")

def branch_and_bound_kmer_leaderboard(sequence, k):
    freq = defaultdict(int)

    def backtrack(start, current):
        # BOUND 1: length exceeded
        if len(current) > k:
            return

        # BOUND 2: invalid amino acid
        if any(ch not in valid_amino_acids for ch in current):
            return

        # If k-mer completed
        if len(current) == k:
            freq[current] += 1
            return

        # BOUND 3: insufficient characters left
        if start >= len(sequence):
            return

        # BRANCH: extend substring
        backtrack(start + 1, current + sequence[start])

    # Start from every index
    for i in range(len(sequence)):
        backtrack(i, "")

    # Sort leaderboard by frequency (descending)
    leaderboard = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    return leaderboard


# Example run
sequence = "MALWMRLLPLLALLALWGPDPAAA"  # curated UniProt sequence
k = 3

leaderboard = branch_and_bound_kmer_leaderboard(sequence, k)

print("Leaderboard of consistent k-mers:")
print("Rank\tK-mer\tFrequency")
for rank, (kmer, count) in enumerate(leaderboard, start=1):
    print(f"{rank}\t{kmer}\t{count}")


Leaderboard of consistent k-mers:
Rank	K-mer	Frequency
1	ALW	2
2	LLA	2
3	LAL	2
4	MAL	1
5	LWM	1
6	WMR	1
7	MRL	1
8	RLL	1
9	LLP	1
10	LPL	1
11	PLL	1
12	ALL	1
13	LWG	1
14	WGP	1
15	GPD	1
16	PDP	1
17	DPA	1
18	PAA	1
19	AAA	1
